In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import threading

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier


In [16]:
trains= ["University2/university_ps_dcexpanded_new_train.csv" , "University1/university_saptarshi_train.csv", "Company/it_train_new1.csv"]
tests = ["change_University2.csv","change_University1.csv","change_Company.csv"]
test1 = ["change2_University2.csv","change2_University1.csv","change2_Company.csv"]
data_names = ["University2","University1","Company"]

models_dict = {
    #"SupportVectorMachine": SVC(kernel="linear"),
    #"LogisticRegression": LogisticRegression(solver="saga", n_jobs=-1),
    "ArtificalNeuralNetwork": MLPClassifier(hidden_layer_sizes=30, max_iter=1500),
    "DecisionTree": DecisionTreeClassifier(),
    #"ExtraTree": ExtraTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=-1),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="error", n_jobs=-1),
    #"LightGBM": LGBMClassifier(n_estimators=100, n_jobs=-1),
    #"AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=1.0),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),
}

In [17]:
def GetAttributeMapping(data,grp=None,grp_gap=20,map_type=1):
    #print('in mapping')
    #print(data)
    #for col in data.columns[:7]:
    #        print(col,data[col].unique())
    mapping = {}
    mapping['NotA'] = -1
    mapping['NA'] = -1
    mapping[0] = 0
    mapping[-1] = -1
    mapping['Yes'] = 1
    mapping['No'] = 0
    mapping['YES'] =1
    mapping['NO'] = 0
    if(map_type==1):
        for col in data.columns[:len(data.columns)-1]:
            if col != 'Year' or col !='Year.1':
                col_un = data[col].unique()
                cnt = 1
                for val in col_un:
                    if(val != 'NotA' or val != 'NA' ):
                        mapping[val] = cnt
                        cnt = cnt + 1
            else:
                for val in col_un:
                    if(val != 'NotA' or val != 'NA' ):
                        mapping[val] = int(val)
        return mapping
    

In [18]:
def same_conditions(col1,col2):
    if ((col1 == -1) or (col2 == -1)):
        return 2
    elif (col1==col2):
        return 1
    else:
        return 0

def chk_nota(col):
    if(col==-1):
        return 1
    else:
        return 0

In [19]:
class PrepareDataARFE:      
    def GetPreparedDataUni2(train_data,test_data):  
        data = pd.concat([train_data,test_data],axis=0)
        mapping = GetAttributeMapping(data,map_type=1)
        data_encoded = data.replace(mapping)
        data_encoded['sameCourse'] = data_encoded.apply(lambda x: same_conditions(x['Course'], x['Course.1']), axis=1)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded['sameDeg'] = data_encoded.apply(lambda x: same_conditions(x['Degree'], x['Degree.1']), axis=1)
        data_encoded['sameYr'] = data_encoded.apply(lambda x: same_conditions(x['Year'], x['Year.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Course',axis=1)
        data_encoded = data_encoded.drop('Course.1',axis=1)
        data_encoded = data_encoded.drop('Degree',axis=1)
        data_encoded = data_encoded.drop('Degree.1',axis=1)
        data_encoded = data_encoded.drop('Year',axis=1)
        data_encoded = data_encoded.drop('Year.1',axis=1)
        X = data_encoded.loc[:, data_encoded.columns != 'Access']
        y = data_encoded.loc[:, data_encoded.columns == 'Access']
        X_train = X.head(train_data.shape[0])
        X_test = X.tail(test_data.shape[0])
        y_train = y.head(train_data.shape[0])
        y_test = y.tail(test_data.shape[0])
        #print(y_train['Access'].unique(),y_test['Access'].unique())
        return X_train, X_test, y_train, y_test
    def GetPreparedDataUni1(train_data,test_data):  
        data = pd.concat([train_data,test_data],axis=0)
        mapping = GetAttributeMapping(data,map_type=1)
        data_encoded = data.replace(mapping)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded['sameDeg'] = data_encoded.apply(lambda x: same_conditions(x['Degree'], x['Degree.1']), axis=1)
        data_encoded['sameYr'] = data_encoded.apply(lambda x: same_conditions(x['Year'], x['Year.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Degree',axis=1)
        data_encoded = data_encoded.drop('Degree.1',axis=1)
        data_encoded = data_encoded.drop('Year',axis=1)
        data_encoded = data_encoded.drop('Year.1',axis=1)
        X = data_encoded.loc[:, data_encoded.columns != 'Access']
        y = data_encoded.loc[:, data_encoded.columns == 'Access']
        X_train = X.head(train_data.shape[0])
        X_test = X.tail(test_data.shape[0])
        y_train = y.head(train_data.shape[0])
        y_test = y.tail(test_data.shape[0])
        return X_train, X_test, y_train, y_test

    def GetPreparedDataIT(train_data,test_data):
        data = pd.concat([train_data,test_data],axis=0)
        mapping = GetAttributeMapping(data,map_type=1)
        data_encoded = data.replace(mapping)
        data_encoded['sameProj'] = data_encoded.apply(lambda x: same_conditions(x['Project_name'], x['Project_Name']), axis=1)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Project_name',axis=1)
        data_encoded = data_encoded.drop('Project_Name',axis=1)
        X = data_encoded.loc[:, data_encoded.columns != 'Access']
        y = data_encoded.loc[:, data_encoded.columns == 'Access']        
        X_train = X.head(train_data.shape[0])
        X_test = X.tail(test_data.shape[0])
        y_train = y.head(train_data.shape[0])
        y_test = y.tail(test_data.shape[0])
        #print(y_train['Access'].unique(),y_test['Access'].unique())
        return X_train, X_test, y_train, y_test

In [20]:
test_data = pd.read_csv(tests[2])
test_data.describe()
test_data.head()

,DESIGNATION,Project_name,Department,Resource,Project_Name,Department.1,Access
0,CEO,NotA,ADMINISTRATION,SALARY_DETAIL,P3,HR,YES
1,CEO,NotA,ADMINISTRATION,CLIENT_DETAIL,P3,HR,YES
2,CEO,NotA,ADMINISTRATION,EMP_DETAIL,P3,DEVELOPMENT,YES
3,CEO,NotA,ADMINISTRATION,PROJECT_DETAIL,P3,DEVELOPMENT,YES
4,CEO,NotA,ADMINISTRATION,DATABASE,P3,TECHNICAL,YES


In [21]:
import pandas as pd
import time
from sklearn.metrics import confusion_matrix

def run_models(train,test, models_dict,i,name=''):
    train_data = pd.read_csv(train[i])
    test_data = pd.read_csv(test[i])
    train_data.fillna('NotA', inplace=True)
    test_data.fillna('NotA', inplace=True)
    if(i==2):
        X_train, X_test, y_train, y_test = PrepareDataARFE.GetPreparedDataIT(train_data,test_data)
       
    elif(i==1):
        X_train, X_test, y_train, y_test = PrepareDataARFE.GetPreparedDataUni1(train_data,test_data)
    else:
        X_train, X_test, y_train, y_test = PrepareDataARFE.GetPreparedDataUni2(train_data,test_data)
    for model_name, model in models_dict.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
        print("")
        with open(f'{data_names[i]}_arfe'+name+'_results.txt', 'a') as f:
            f.write(f"Model: {model_name}\n")
            f.write(f"Train Accuracy: {model.score(X_train, y_train)}\n")
            f.write(f"Test Accuracy: {model.score(X_test, y_test)}\n")
            f.write(f"Confusion Matrix: {cm}\n")
            f.write("\n")
        
        
run_models(trains,tests,models_dict,0,"_change")
run_models(trains,tests,models_dict,1,"_change")
run_models(trains,tests,models_dict,2,"_change")
run_models(trains,test1,models_dict,0,'_change2')
run_models(trains,test1,models_dict,1,'_change2')
run_models(trains,test1,models_dict,2,'_change2')





C:\Users\tipsh\AppData\Local\Temp\ipykernel_8856\2612894688.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[73  0]
 [19 73]]

[[73  0]
 [19 73]]



C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[[73  0]
 [11 81]]



C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


...

C:\Users\tipsh\AppData\Local\Temp\ipykernel_8856\2612894688.py:50: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[ 7  1]
 [ 5 27]]

[[ 6  2]
 [ 8 24]]

[[ 6  2]
 [ 5 27]]



C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[[ 6  2]
 [ 6 26]]

[[ 6  2]
 [ 8 24]]

[[ 6  2]
 [ 5 27]]



C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
